# Create secrets.yaml

Expected fields (no quotes):

```yaml
client_id: 123
client_secret: abc
client_key: abc
```

# Using the interface:

## Construct an API query

A query is a combination of a "type (and, or, not)" with multiple Conditions ("Cond")

Each condition is a combination of a "field" (Fields, F), "value" and a operation ("Operations", "Op").

In [13]:
from tiktok_research_api_lib.query import Query, Cond, Fields, Op

query = Query(
        and_=[
            Cond(Fields.hashtag_name, "garfield", Op.EQ),
            Cond(Fields.region_code, "US", Op.EQ),

            # Alternative version with multiple countries - Then the operation changes to "IN" instead of "EQ" (equals) as it's a list
            # the library handles list vs str natively
            # Cond(Fields.region_code, ["US", "UK"], Op.IN),
        ],
    )

## TikTokApiClient provides a high-level interface to fetch all api results, and optionally store them in a database

In [17]:
from pathlib import Path
from datetime import datetime
from tiktok_research_api_lib.query import Query, Cond, Fields, Op
from tiktok_research_api_lib.api_client import ApiClientConfig, TikTokApiClient

config = ApiClientConfig(query=query,
                         start_date=datetime.fromisoformat("2024-03-01"),
                         end_date=datetime.fromisoformat("2024-03-02"),
                         engine=None,
                         api_credentials_file=Path("./secrets.yaml"))
api_client = TikTokApiClient.from_config(config)

# api_results_iter yields each API reponse as a parsed TikTokApiClientFetchResult.
# Iteration stops when the API indicates the query results have been fully delivered
[result for result in api_client.api_results_iter()]

[TikTokApiClientFetchResult(videos=[{'username': 'garfballz', 'video_description': 'recent travels #garf #garfield #curced #travel ', 'view_count': 78, 'comment_count': 1, 'create_time': 1709422791, 'hashtag_names': ['travel', 'garfield', 'garf', 'curced'], 'like_count': 32, 'share_count': 0, 'id': 7341914795693690143, 'music_id': 7278879865305238318, 'region_code': 'US'}, {'music_id': 7239357525690288129, 'region_code': 'US', 'username': 'zayda_cake', 'video_description': 'No regrets  #acrossthespiderverse #atsv #miguelohara #GabrielOhara #fanart #garfield ', 'hashtag_names': ['atsv', 'fanart', 'garfield', 'miguelohara', 'acrossthespiderverse', 'gabrielohara'], 'id': 7341912592404778283, 'like_count': 40, 'view_count': 18, 'comment_count': 0, 'create_time': 1709422258, 'share_count': 0}, {'view_count': 68, 'create_time': 1709420417, 'hashtag_names': ['eminem', 'bojackhorseman', 'slideshow', 'garfield', 'fyp', 'jessepinkman', 'badger', 'jimhalpert', 'toddchavez', 'fypage'], 'id': 73419

In [16]:
# fetch_all fetches all API results and returns a single TikTokApiClientFetchResult with all API results.
api_client.fetch_all()

TikTokApiClientFetchResult(videos=[{'music_id': 7320266683242171168, 'region_code': 'US', 'username': 'skyefoox', 'view_count': 87, 'comment_count': 1, 'like_count': 11, 'id': 7341544774308138286, 'share_count': 0, 'video_description': '#orange #ahsoka #tonythetiger #goofy #tigger #velma #theflintstones #garfield #aang #foryou #foryoupage #fypageシ #fypage #fyp #fy #slideshow #goviral', 'create_time': 1709336617, 'hashtag_names': ['orange', 'goofy', 'slideshow', 'foryou', 'goviral', 'fy', 'tigger', 'garfield', 'fyp', 'aang', 'velma', 'theflintstones', 'tonythetiger', 'ahsoka', 'foryoupage', 'fypage', 'fypageシ']}, {'id': 7341539573907918122, 'like_count': 12, 'view_count': 2, 'video_description': 'At the brass armadillo Phoenix #phoenixpicker #case160 #foryoupageofficiall #brassarmadillophoinex #yeeyeenation #foryoupage❤️❤️❤️foryou💞💞💜 #disneysnowwhite #brassarmadillo #springbreak #garfield #garfiledthecat #garfielstuff #vinatgeaesthetic #viralvideo #video #viral #vinatgeaesthetic #arizon

In [4]:
# If you provide a SqlAlchemy engine in the ApiClientConfig you can use TikTokApiClient to store results as they are received
api_client.fetch_all(store_results_after_each_response=True) # or equivalent function fetch_and_store_all()

## TikTokApiRequestClient and TikTokRequest provide a lower-level interface to API

In [1]:
from pathlib import Path
from tiktok_research_api_lib.api_client import TikTokApiRequestClient, TiktokRequest

# reads from secrets.yaml in the same directory
request_client = TikTokApiRequestClient.from_credentials_file(Path("./secrets.yaml"))

In [2]:
from tiktok_research_api_lib.query import Query, Cond, Fields, Op

In [3]:
# sample query
req = TiktokRequest(
    query=Query(or_=Cond(Fields.video_id, ["7345557461438385450", "123456"], Op.IN)),
    start_date="20240301",
    end_date="20240329",
)

req

TiktokRequest(query=Query(and_=None, or_=[Condition(field=_Field(name='video_id', validator=<instance_of validator for type <class 'str'>>), field_values=['7345557461438385450', '123456'], operation=<Operations.IN: 'IN'>)], not_=None), start_date='20240301', end_date='20240329', max_count=100, is_random=False, cursor=None, search_id=None)

In [4]:
request_client.fetch(req)

TikTokResponse(data={'search_id': '7353702326177600558', 'videos': [{'create_time': 1710270898, 'hashtag_names': [], 'like_count': 21140, 'region_code': 'US', 'share_count': 1164, 'video_description': 'Friends who nap together>>>>>> Watch the Peanuts Classics on Apple TV+! ', 'view_count': 44804, 'comment_count': 2069, 'id': 7345557461438385450, 'music_id': 7345557575716440875, 'username': 'snoopy'}], 'cursor': 1, 'has_more': False}, videos=[{'create_time': 1710270898, 'hashtag_names': [], 'like_count': 21140, 'region_code': 'US', 'share_count': 1164, 'video_description': 'Friends who nap together>>>>>> Watch the Peanuts Classics on Apple TV+! ', 'view_count': 44804, 'comment_count': 2069, 'id': 7345557461438385450, 'music_id': 7345557575716440875, 'username': 'snoopy'}])

In [5]:
hashtags = ["fyp", "foryou", "foryoupage"]

# Multiple conditions
req = TiktokRequest(
    query=Query(
        and_=[
            Cond(Fields.hashtag_name, hashtags, Op.IN),
            Cond(Fields.region_code, "US", Op.EQ),

            # Alternative version with multiple countries - Then the operation changes to "IN" instead of "EQ" (equals) as it's a list
            # the library handles list vs str natively
            # Cond(Fields.region_code, ["US", "UK"], Op.IN),

        ]
    ),
    start_date="20220101",
    end_date="20220129",
)

request_client.fetch(req)

TikTokResponse(data={'cursor': 100, 'has_more': True, 'search_id': '7353702326177633326', 'videos': [{'create_time': 1643500799, 'hashtag_names': ['foryou', 'newtrend', 'elfitup', 'virall', 'followformorevideo', 'heytiktok', 'fypシ', 'CloseYourRings', 'ArbysDiabloDare', 'ITriedItIPrimedIt'], 'music_id': 6895258046780541701, 'video_description': 'Yeahyeahyeah🤣😋#ITriedItIPrimedIt #fypシ #followformorevideo #foryou #ArbysDiabloDare #elfitup #CloseYourRings #heytiktok #virall #newtrend', 'region_code': 'US', 'share_count': 2, 'username': 'trxpdoll._.niyaa', 'view_count': 484, 'comment_count': 1, 'effect_ids': ['0'], 'id': 7058782183653133614, 'like_count': 56}, {'hashtag_names': ['masterchef', 'fyp', 'foryoupage', 'cheftiktok', 'chefskiss'], 'region_code': 'US', 'share_count': 0, 'username': 'cr33psh0w', 'video_description': 'what’s in y’all’s fridge? #foryoupage #fyp #masterchef #cheftiktok #chefskiss', 'view_count': 298, 'comment_count': 0, 'effect_ids': ['0'], 'like_count': 12, 'music_id'